In [1]:
%load_ext autoreload 
%autoreload 2

import os
import gpflow
import numpy as np
import scipy.io

import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use('ggplot')
mpl.rcParams['pgf.rcfonts'] = False

os.environ["CUDA_VISIBLE_DEVICES"] = '0'

from gp_svgp import GP_MNIST_SVGP
from mnist_cnn import CNN_MNIST
from mnist_svm import SVM_MNIST
from hybrid_mnist import Hybrid_MNIST
from mnist_preprocessing import get_mnist_data, preprocess_images

#from plotting import Plotter

Using TensorFlow backend.


In [2]:
class Data(object):
    def __init__(self):
        self.data = {}
    def write(self, path, value):
        ptr = self.data
        for key in path[:-1]:
            if key not in ptr:
                ptr[key] = {}
            ptr = ptr[key]
        ptr[path[-1]] = value

data = Data()

In [3]:
# Standard MNIST test data
((_, mnist_test_imgs), (ys_train_onehot, ys_test_onehot)) = get_mnist_data()
mnist_train_features = np.genfromtxt("../data/mnist_train_features.csv", delimiter=",")
mnist_test_features = np.genfromtxt("../data/mnist_test_features.csv", delimiter=",")

ys_train = np.argmax(ys_train_onehot, axis=1)
ys_test = np.argmax(ys_test_onehot, axis=1)
# easier to remember handle
correct_classes = ys_test
n = len(ys_test)

In [4]:
cnn_model = CNN_MNIST(isolate=True) # isolate sessions

In [5]:
svm_model = SVM_MNIST(mnist_train_features, ys_train, nb_classes=10)

In [6]:
hybrid_models = []
for tol in np.arange(0, 2.0, 0.5):
    hybrid_models.append(Hybrid_MNIST(accept_cnn_tolerance = tol))
for tol in np.arange(0, 2.0, 0.5):
    hybrid_models.append(Hybrid_MNIST(accept_cnn_tolerance = tol, stronger_criterion=True))

In [7]:
# instantiate models we want to test
# only one at a time here to avoid interference

get_gp_model = [lambda: GP_MNIST_SVGP(kernel=gpflow.kernels.Linear, 
                         whitevar=0.1,
                         latent_split=25,
                         whiten=True,
                         q_diag=True,
                         minibatch=8000,
                         whitevar_trainable=False,
                         feat_trainable=True,
                         name="Linear Kernel GP"),
                lambda: GP_MNIST_SVGP(kernel=gpflow.kernels.Matern12, 
                         whitevar=0.1,
                         latent_split=25,
                         whiten=True,
                         q_diag=True,
                         minibatch=8000,
                         whitevar_trainable=False,
                         feat_trainable=True,
                         name="Matern12 Kernel GP"),
                lambda: GP_MNIST_SVGP(kernel=gpflow.kernels.Matern32, 
                         whitevar=0.1,
                         latent_split=25,
                         whiten=True,
                         q_diag=True,
                         minibatch=8000,
                         whitevar_trainable=False,
                         feat_trainable=True,
                         name="Matern32 Kernel GP"),
                lambda: GP_MNIST_SVGP(kernel=gpflow.kernels.Matern32, 
                         whitevar=0.1,
                         latent_split=25,
                         whiten=True,
                         q_diag=True,
                         minibatch=8000,
                         whitevar_trainable=False,
                         feat_trainable=True,
                         name="Matern52 Kernel GP"),
                lambda: GP_MNIST_SVGP(kernel=gpflow.kernels.RBF, 
                         whitevar=0.1,
                         latent_split=25,
                         whiten=True,
                         q_diag=True,
                         minibatch=8000,
                         whitevar_trainable=False,
                         feat_trainable=True,
                         name="RBF Kernel GP")
               ]


In [8]:
record_in = 'Standard MNIST'

# prediction accuracies
subkey = 'accuracy'


#CNN
mnist_cnn_preds = cnn_model.predict_img_batch(mnist_test_imgs)
mnist_cnn_classes = np.argmax(mnist_cnn_preds, axis=1)
mnist_cnn_correct = correct_classes == mnist_cnn_classes
mnist_cnn_accuracy = np.count_nonzero(mnist_cnn_correct) / n
print("CNN accuracy: ", mnist_cnn_accuracy)
data.write([record_in, subkey, cnn_model.name],mnist_cnn_accuracy)

# SVM
mnist_svm_classes = svm_model.predict_batch(mnist_test_features)
mnist_svm_correct = correct_classes == mnist_svm_classes
mnist_svm_accuracy = np.count_nonzero(mnist_svm_correct) / n
print("SVM accuracy: ", mnist_svm_accuracy)
data.write([record_in, subkey, svm_model.name], mnist_svm_accuracy)


CNN accuracy:  0.9911
SVM accuracy:  0.9937


In [ ]:
# GP
for gp_model_getter in get_gp_model:
    gp_model = gp_model_getter()
    print("Instantiated", gp_model.name)
    mnist_gp_mu, mnist_gp_var = gp_model.predict_batch(mnist_test_features)
    mnist_gp_classes = np.argmax(mnist_gp_mu, axis=1)
    mnist_gp_correct = correct_classes == mnist_gp_classes
    mnist_gp_accuracy = np.count_nonzero(mnist_gp_correct) / n
    print(gp_model.name, "accuracy: ", mnist_gp_accuracy)
    data.write([record_in, subkey, gp_model.name], mnist_gp_accuracy)

    for hybrid in hybrid_models:
        name = hybrid.name + "(GP: {})".format(gp_model.name)
        hybrid_pred, hybrid_mus, hybrid_vars = hybrid.combine_predictions(mnist_cnn_preds, mnist_gp_mu, mnist_gp_var)
        hybrid_classes = hybrid_pred[:, 1]
        hybrid_correct = correct_classes == hybrid_classes
        hybrid_accuracy = np.count_nonzero(hybrid_correct) / n
        print(name, "accuracy:", hybrid_accuracy)
        data.write([record_in, subkey, name], hybrid_accuracy)

Initialized Linear Kernel GP
Training GP from scratch


In [ ]:
# ---- White Noise + NMNIST ----

# Preprocess the N-MNIST data just like standard MNIST
nmnist_noisy = scipy.io.loadmat('../data/raw/n-mnist/nmnist-awgn.mat')

awgn_x_test = preprocess_images(nmnist_noisy['test_x'])
awgn_y_test_onehot = nmnist_noisy['test_y']
awgn_y_test = np.argmax(awgn_y_test_onehot, axis=1)
awgn_correct_classes = awgn_y_test

In [ ]:
record_in = 'MNIST + AWGN'
subkey = 'Accuracy'

awgn_feats = cnn_model.extract_features(awgn_x_test)

#CNN
awgn_cnn_preds = cnn_model.predict_img_batch(awgn_x_test)
awgn_cnn_classes = np.argmax(awgn_cnn_preds, axis=1)
awgn_cnn_correct = awgn_correct_classes == awgn_cnn_classes
awgn_cnn_accuracy = np.count_nonzero(awgn_cnn_correct) / n
print("CNN accuracy: ", awgn_cnn_accuracy)
data.write([record_in, subkey, cnn_model.name], awgn_cnn_accuracy)

# SVM
awgn_svm_classes = svm_model.predict_batch(awgn_feats)
awgn_svm_correct = awgn_correct_classes == awgn_svm_classes
awgn_svm_accuracy = np.count_nonzero(awgn_svm_correct) / n
print("SVM accuracy: ", awgn_svm_accuracy)
data.write([record_in, subkey, svm_model.name], awgn_svm_accuracy)

In [ ]:
nmnist_blur = scipy.io.loadmat('../data/raw/n-mnist/nmnist-blur.mat')
blur_x_test = preprocess_images(nmnist_blur['test_x'])
blur_y_test_onehot = nmnist_blur['test_y']
blur_y_test = np.argmax(blur_y_test_onehot, axis=1)
blur_correct_classes = blur_y_test

blur_feats = cnn_model.extract_features(blur_x_test)

In [ ]:
record_in = 'MNIST + blur'
subkey = 'Accuracy'

blur_feats = cnn_model.extract_features(blur_x_test)

#CNN
blur_cnn_preds = cnn_model.predict_img_batch(blur_x_test)
blur_cnn_classes = np.argmax(blur_cnn_preds, axis=1)
blur_cnn_correct = blur_correct_classes == blur_cnn_classes
blur_cnn_accuracy = np.count_nonzero(blur_cnn_correct) / n
print("CNN accuracy: ", blur_cnn_accuracy)
data.write([record_in, subkey, cnn_model.name], blur_cnn_accuracy)

# SVM
blur_svm_classes = svm_model.predict_batch(blur_feats)
blur_svm_correct = blur_correct_classes == blur_svm_classes
blur_svm_accuracy = np.count_nonzero(blur_svm_correct) / n
print("SVM accuracy: ", blur_svm_accuracy)
data.write([record_in, subkey, svm_model.name], blur_svm_accuracy)

In [ ]:
nmnist_noisy_lowcontrast = scipy.io.loadmat('../data/raw/n-mnist/nmnist-contrast.mat')
record_in = 'Low Contrast MNIST'
if record_in not in data:
    data[record_in] = {} 
# noisy
subkey = 'Accuracy'
if subkey not in data[record_in]:
    data[record_in][subkey] = {}
    
lowcontrast_feats = mnist_cnn.extract_features(nmnist_noisy_lowcontrast)